In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
from nilearn.plotting import plot_design_matrix, plot_stat_map, view_img
from nilearn.reporting import get_clusters_table
from nilearn import datasets
import glob
import os
import numpy as np
import pandas as pd

In [ ]:
def get_filt_tval_img(reg, reg_rt = "1", mnum = "1", mname = 'overall_mean', tstat="1", threshold=0.95):
    
    level3_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/derivatives/nilearn/glm/level3/'
    model_path = 'model'+mnum+'_reg-rt'+reg_rt
    img_path = os.path.join(level3_path, model_path, mname, reg+'_'+model_path)
    
    pos_pval_fn = 'rand_tfce_corrp_%s_%s_model%s_reg-rt%s_tstat%s.nii.gz'%(mname, reg, mnum, reg_rt, tstat)
    neg_pval_fn = 'rand_tfce_corrp_%s_%s_model%s_reg-rt%s_tstat%s.nii.gz'%(mname, 'neg_'+reg, mnum, reg_rt, tstat)
    tval_fn = 'rand_tfce_%s_%s_model%s_reg-rt%s_tstat%s.nii.gz'%(mname, reg, mnum, reg_rt, tstat)
    
    pos_pval_img = os.path.join(img_path, pos_pval_fn)
    neg_pval_img = os.path.join(img_path, neg_pval_fn)
    tval_img = os.path.join(img_path, tval_fn)
    
    pos_pval_img = nib.load(pos_pval_img)
    neg_pval_img = nib.load(neg_pval_img)
    tval_img = nib.load(tval_img)
    
    tval_data = tval_img.get_fdata()
    pos_pval_data = pos_pval_img.get_fdata()
    neg_pval_data = neg_pval_img.get_fdata()
    
    filt_tval_data = np.where(pos_pval_data > threshold, tval_data, np.where(neg_pval_data > threshold, tval_data, 0))
    filt_tval_img = nib.Nifti1Image(filt_tval_data.astype(np.float64), tval_img.affine)
    
    return filt_tval_img

def plot_filt_tval_img(reg, reg_rt = "1", mnum = "1", mname = 'overall_mean', tstat="1", threshold=0.95, interactive=False, cut_coords = None, display_mode = 'ortho', draw_cross=False):
    filt_tval_img = get_filt_tval_img(reg=reg, reg_rt = reg_rt, mnum = mnum, mname = mname, tstat=tstat, threshold=threshold)
    title='%s_model%s_reg-rt%s'%(reg, mnum, reg_rt)
    
    if(len(np.unique(filt_tval_img.get_fdata())) == 1):
        print('Nothing survives correction for %s, model = %s, tstat = %s, reg_rt = %s at threshold p < %s'%(reg, mnum, tstat, reg_rt, str(round(1-threshold, 2))))
    elif interactive:
        view = view_img(filt_tval_img, 
             draw_cross=draw_cross,
             title=title, cut_coords = cut_coords)
        
        return view
    else:
        print('Plotting tvalues filtered for corrected p values < %s'%str(round(1-threshold, 2)))
        plot_stat_map(filt_tval_img, 
             draw_cross=draw_cross,
             title=title, cut_coords = cut_coords, display_mode = display_mode)

        
def get_filt_diff_tval_img(reg, reg_rt1, reg_rt2, mnum1, mnum2, mname, tstat="1", threshold=0.95):
    
    img1 = get_filt_tval_img(reg=reg, reg_rt = reg_rt1, mnum = mnum1, mname = mname, tstat=tstat, threshold=threshold)
    img2 = get_filt_tval_img(reg=reg, reg_rt = reg_rt2, mnum = mnum2, mname = mname, tstat=tstat, threshold=threshold)
    
    img1_data = img1.get_fdata()
    img2_data = img2.get_fdata()
    filt_diff_tval_data = img1_data - img2_data
    filt_diff_tval_img = nib.Nifti1Image(filt_diff_tval_data.astype(np.float64), img1.affine)
    
    return filt_diff_tval_img
        
def plot_filt_diff_tval_img(reg, reg_rt1, reg_rt2, mnum1, mnum2, mname = 'overall_mean', tstat="1", threshold=0.95, disp_threshold = 2, interactive=False, cut_coords = None, display_mode = 'ortho', draw_cross=False):
    filt_diff_tval_img = get_filt_diff_tval_img(reg=reg, reg_rt1 = reg_rt1, reg_rt2 = reg_rt2, mnum1 = mnum1, mnum2 = mnum2, mname = mname, tstat=tstat, threshold=threshold)
    title='%s model%s_reg-rt%s - model%s_reg-rt%s '%(reg, mnum1, reg_rt1, mnum2, reg_rt2)
    
    if(len(np.unique(filt_diff_tval_img.get_fdata())) == 1):
        print('Nothing survives correction for the difference image %s, model = %s, tstat = %s, reg_rt = %s at threshold p < %s'%(reg, mnum, tstat, reg_rt, str(round(1-threshold, 2))))
    
    elif interactive:
        view = view_img(filt_diff_tval_img, 
             draw_cross=draw_cross,
             title=title, cut_coords = cut_coords, threshold = disp_threshold)
        
        return view
    else:
        print('Plotting tvalues filtered for corrected p values < %s'%str(round(1-threshold, 2)))
        plot_stat_map(filt_diff_tval_img, 
             draw_cross=draw_cross,
             title=title, cut_coords = cut_coords, display_mode = display_mode, threshold = disp_threshold)

        
def get_mean_cor_df(reg_rt = "1", mnum = "1"):
    
    level3_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/derivatives/nilearn/glm/level3/'
    model_path = 'model'+mnum+'_reg-rt'+reg_rt
    df_path = os.path.join(level3_path, model_path, model_path+'_mean_desmat_cor.csv')
    
    mean_cor_df = pd.read_csv(df_path)
    
    return(mean_cor_df)

# Regressor types

- ev = event; amplitude = 1  
- par = parametric; amplitude = demeaned    
- st = stick (0 duration); amplitude = 1 
- rt = reaction time; amplitude = demeaned RT

Citing Grinband et al. (2008) people agree that reaction times must be taken into account but model them in different ways. Schonberg, Poldrack et al. papers use overall response time for event and parametric regressor as well as a parametric RT regressor. Many in the rest of the decision making literature use boxcars with RT durations (e.g. Dunsmoor et al. (2011), Chang et al. (2011), Chang and Sanfey (2013), Shuck et al. (2016)). 

Since it is common to use boxcars with RT duration and I don't know others who model RT the way Poldrack et al. do I will present those results below.

# Model 1: fractalProb_ev + stim_ev + choiceShift_st + reward_ev

Correlation between regressors averaged across subjects and runs

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "1")

In [ ]:
reg = 'fractalProb_ev'
mnum = '1'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

In [ ]:
reg = 'stim_ev'
mnum = '1'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)


Check for left motor cortex activity. Cross is the precentral gyrus peak reported in Hare et al.

In [ ]:
reg = 'choiceShift_st'
mnum = '1'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)

Motor cortex activity is a little ventral than expected but more consistent for choiceShift regressor. It is, however, not lateralized to the left.

In [ ]:
reg = 'reward_ev'
mnum = '1'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat,)

# Model 1a: fractalProb_ev + stim_ev + choiceShiftLeft_st + choiceShiftRight_st + reward_ev

In [ ]:
reg = 'choiceShift_st'
mnum = '1'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)


In [ ]:
reg = 'choiceShiftLeft_st'
mnum = '1a'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)

In [ ]:
reg = 'choiceShiftRight_st'
mnum = '1a'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)


# Model 2: Model 1 + fractalProb_par

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "2")

In [ ]:
reg = 'fractalProb_ev'
mnum = '2'
mname = 'overall_mean'
tstat = '1'
cut_coords = (1, 29, -14)

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1="0", reg_rt2="0", mnum1="1", mnum2="2", cut_coords = cut_coords)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'stim_ev'
mnum = '2'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'choiceShift_st'
mnum = '2'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords = cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '2'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '2'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

# Model 3: Model 2 + valDiff_par

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "3")

In [ ]:
reg = 'fractalProb_ev'
mnum = '3'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'stim_ev'
mnum = '3'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)
print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))


In [ ]:
reg = 'choiceShift_st'
mnum = '3'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords = cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '3'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)
print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))


In [ ]:
reg = 'fractalProb_par'
mnum = '3'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum)

print('No interesting change from Model 2 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valDiff_par'
mnum = '3'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

plot_filt_diff_tval_img(reg, reg_rt1="0", reg_rt2="1", mnum1=mnum, mnum2=mnum, cut_coords=cut_coords)

# Model 4: Model 2 + valChosen + valUnchosen

The added value regressors are not associated with activity anywhere BUT they change the maps for some other regressors.

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "4")

In [ ]:
reg = 'fractalProb_ev'
mnum = '4'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

- Decrease in vStr in Model 4 compared to Model 1
- Increase in vmPFC in Model 4 compared to Model 1

In [ ]:
reg = 'stim_ev'
mnum = '4'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
plot_filt_diff_tval_img(reg, reg_rt1 =reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

In [ ]:
reg = 'choiceShift_st'
mnum = '4'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords = cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '4'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '4'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (33, -75, 31)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "2", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum, cut_coords = cut_coords)

print('No interesting change from Model 2 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valChosen_par'
mnum = '4'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

reg_rt = '1'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

In [ ]:
reg = 'valUnchosen_par'
mnum = '4'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

reg_rt = '1'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

# Model 5: Model 2 + valDiffLottery + valDiffFractal

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "5")

In [ ]:
get_mean_cor_df(reg_rt = "1", mnum = "5")

In [ ]:
reg = 'fractalProb_ev'
mnum = '5'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

The frontal changes apply to the RT regressor when RT is modeled separately

In [ ]:
reg = 'stim_ev'
mnum = '5'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (3, 59, 0)

# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords= cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords= cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords= cut_coords)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'choiceShift_st'
mnum = '5'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords= cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '5'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '5'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valDiffLottery_par'
mnum = '5'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)


In [ ]:
reg = 'valDiffFractal_par'
mnum = '5'
mname = 'overall_mean'
tstat = '1'
threshold = 0.95

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, threshold=threshold)


# Model 6: Model 2 + valChosenLottery + valUnchosenLottery + valChosenFractal + valUnchosenFractal

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "6")

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "6")

In [ ]:
reg = 'fractalProb_ev'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (-31, -68, -40)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords = cut_coords)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'stim_ev'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (0, -32, 12)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords = cut_coords)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'choiceShift_st'
mnum = '6'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords= cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum)

print('No interesting change from Model 2 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valChosenLottery_par'
mnum = '6'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-1, 9, 0)

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)

In [ ]:
reg = 'valUnchosenLottery_par'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

In [ ]:
reg = 'valChosenFractal_par'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

In [ ]:
reg = 'valUnchosenFractal_par'
mnum = '6'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

# Model 7: Model 5 + reward_par

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "7")

In [ ]:
reg = 'fractalProb_ev'
mnum = '7'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (-28, -66, -42)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'stim_ev'
mnum = '7'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (0, -32, 12)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'choiceShift_st'
mnum = '7'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords= cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '7'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (23, 0, 10)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords = cut_coords)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '7'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum)

print('No interesting change from Model 2 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valDiffLottery_par'
mnum = '7'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
cut_coords = (-10, 4, 5)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "5", mname = mname, tstat = tstat, cut_coords = cut_coords)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="5", mnum2=mnum)

In [ ]:
reg = 'valDiffFractal_par'
mnum = '7'
mname = 'overall_mean'
tstat = '1'
# cut_coords = (-1, 9, 0)

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="5", mnum2=mnum)


In [ ]:
reg = 'reward_par'
mnum = '7'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, interactive=True)

# Model 8: Model 5 + rpe_par

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "8")

In [ ]:
reg = 'fractalProb_ev'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'stim_ev'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (0, -32, 12)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'choiceShift_st'
mnum = '8'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords= cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

cut_coords = (-50, 35, 3)

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum)

print('No interesting change from Model 2 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valDiffLottery_par'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
cut_coords = (-10, 4, 5)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "5", mname = mname, tstat = tstat, cut_coords = cut_coords)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="5", mnum2=mnum)


In [ ]:
reg = 'valDiffFractal_par'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
cut_coords = (-10, 4, 5)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "5", mname = mname, tstat = tstat, cut_coords = cut_coords)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="5", mnum2=mnum)


In [ ]:
reg = 'rpe_par'
mnum = '8'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

# Model 9: Model 5 + reward_par + rpe_par

In [ ]:
get_mean_cor_df(reg_rt = "0", mnum = "9")

In [ ]:
get_mean_cor_df(reg_rt = "1", mnum = "9")

In [ ]:
reg = 'fractalProb_ev'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'stim_ev'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# cut_coords = (0, -32, 12)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'choiceShift_st'
mnum = '9'
mname = 'overall_mean'
tstat = '1'
cut_coords = (-45, -34, 64)
draw_cross = True

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords, draw_cross = draw_cross)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum, cut_coords= cut_coords, draw_cross = draw_cross)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'reward_ev'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "1", mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="1", mnum2=mnum)

print('No interesting change from Model 1 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'fractalProb_par'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
# plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="2", mnum2=mnum)

print('No interesting change from Model 2 for %s in Model %s'%(reg, mnum))

In [ ]:
reg = 'valDiffLottery_par'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
cut_coords = (-10, 4, 5)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "5", mname = mname, tstat = tstat, cut_coords = cut_coords)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="5", mnum2=mnum)

In [ ]:
reg = 'valDiffFractal_par'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
cut_coords = (-10, 4, 5)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = "5", mname = mname, tstat = tstat, cut_coords = cut_coords)
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat, cut_coords = cut_coords)
# plot_filt_diff_tval_img(reg, reg_rt1=reg_rt, reg_rt2=reg_rt, mnum1="5", mnum2=mnum)

In [ ]:
reg = 'reward_par'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)

In [ ]:
reg = 'rpe_par'
mnum = '9'
mname = 'overall_mean'
tstat = '1'

reg_rt = '0'
plot_filt_tval_img(reg, reg_rt=reg_rt, mnum = mnum, mname = mname, tstat = tstat)